# Tune angle for GIWAXS
### First steps
Run "make_SAXS_macro.py" or the following cell

In [ ]:
import make_SAXS_macro as mk_macro

start = 0.0     # angle to start sweep at
finish = 1.2    # last angle to include
step = 0.005    # how much angle to change between each exposure
tag = ""        # optional text to add to file name

# create list of angles to scan
angles = mk_macro.arange(start, finish, step)

# create file in Macros directory
mk_macro.create_file(angles, tag)

## Load data set

In [1]:
import numpy as np
# import matplotlib.pylab as plt
import gixspy.plot as plt
import gixspy.loader as load
import gixspy.fitting as fit
%matplotlib qt
# import show_tiff
import tkinter

default_path = "/home/saxs/DATA"

# Create Tk root
root = tkinter.Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

# get list of paths (strings) to each data file
# tiff_list = load.search_tiffs(default_path)
# tiff_list = load.tiff_filenames(
#         "C:\\Users\\Teddy\\OneDrive - UCB-O365\\Rogerslab3\\Teddy\\XRD\\Silicon-silica-TT5-GIWAXS tune 2023-10-10")
tiff_list = load.tiff_filenames(
         "C:\\Users\\Teddy\\OneDrive - UCB-O365\\Rogerslab3\\Teddy\\TPP Films\\Unkown inclusion (benz, tolu, xyl)\\giwaxs\\TT5mm-02_2023-11-06\\tune")
"""
Load data as numpy arrays.
angles = 1D array of motor angle settings
intenisty_data = 3D array of intesnities; indices are (angle_index, z_index, x_index)
direct_beam = 2D array of intensity of the direct beam exposure; indices are (z_index, x_index)
"""
angles, intensity_data, direct_beam = load.tiff_data(tiff_list)

print("angles shape: {}".format(angles.shape))
print("intensity_data shape: {}".format(intensity_data.shape))
print("direct_beam shape: {}".format(direct_beam.shape))

# %gui tk

C:Users\Teddy\OneDrive - UCB-O365\Rogerslab3\Teddy\TPP Films\Unkown inclusion (benz, tolu, xyl)\giwaxs\TT5mm-02_2023-11-06\tune
angles shape: (240,)
intensity_data shape: (240, 1065, 1030)
direct_beam shape: (1065, 1030)


Optional: Display a file

In [ ]:
clip = None         # clip intensity
log_scale = True    # log scale

plt.tiff(direct_beam, clip, log_scale)
plt.show()

### Crop data
Will animate data to confirm the crop

In [2]:
x_pixel_width = 13  # number of pixels to sum across horizontally centered around the beam center
pixels_above = 50   # number of pixels above the beam to keep (for crop)
pixels_below = 20   # number of pixels below the beam to keep (for crop)
pixel_size = 0.075  # mm/pixel

"""Crop data"""
id_c, db_c = load.crop_data(intensity_data, direct_beam, x_pixel_width, pixels_above, pixels_below)
z_px = np.arange(db_c.shape[0])

"""Animate"""
frame_delay = 10    # milliseconds
clip = None         # counts clipping level
log = True          # animate on a log scale

fig, ani = plt.animate(id_c, frame_delay, clip, log)

"""Plot counts vs z vs counts"""
clip = None         # counts clipping level
log = True          # animate on a log scale
plt.plot_all(angles, id_c, pixel_size, clip, log)

print("done")

done


### Find direct beam with gaussian fit

In [7]:
import matplotlib.pylab as plt2
plot = True
db_gfit = fit.direct_beam_line(db_c, plot)

id_l = np.sum(id_c, axis=2)
print(id_l.shape)

db_fit = fit.gaussian0(z_px, *db_gfit).astype(int)
subtract_db = np.copy(id_l)
subtract_db[:, db_fit > 0] = id_l[0, 0]
# subtract_db = id_l - db_fit
# subtract_db[subtract_db < 0] = id_l[0, 0]
print(subtract_db.shape)

for ii in range(len(angles)):
    plt2.plot(subtract_db[ii])

plt2.figure()
threshold = 3000
subtract_db[subtract_db < threshold] = 0
for ii in range(len(angles)):
    plt2.plot(subtract_db[ii])


id_gfit = fit.direct_beam_line(id_c, plot)



Amplitude = 185797.3885831484 counts
Center = 49.37260594967291 pixels
Full width half max = 2.38150696828259 pixels
Total counts in the data = 471003 counts
Total counts in the fit = 471002.9989569331 counts
(240, 70)
(240, 70)


In [5]:
from scipy.optimize import curve_fit

def gaussian_2_fit(x, y):
    mean = np.sum(x * y) / np.sum(y)
    sigma = np.sqrt(np.sum(y * (x - mean) ** 2) / sum(y))
    fwhm = sigma * 2 * np.sqrt(2 * np.log(2))
    params, _ = curve_fit(f=gaussian_2_peak, xdata=x, ydata=y,
                          p0=[np.max(y), 0.5*(np.min(x)+np.max(x)), 2, np.max(y), mean, 2],
                          bounds=((0, np.min(x), 0, 0, np.min(x), 0),
                                  (np.inf, np.max(x), 50, np.inf, np.max(x), 50)))
    return params

def gaussian_2_peak(x, amplitude1, center1, fwhm1, amplitude2, center2, fwhm2):
    return amplitude1 * np.exp(-4 * np.log(2) * ((x - center1) / fwhm1) ** 2) + \
        amplitude2 * np.exp(-4 * np.log(2) * ((x - center2) / fwhm2) ** 2)

def line(intensity, plot=True):
    intensity_l = np.sum(intensity, axis=1)
    z_px = np.arange(len(intensity_l))
    fit_params = gaussian_2_fit(z_px, intensity_l)
    peak_num = 2
    print(fit_params)
    for ii in range(peak_num):
        print(f"Amplitude{ii+1} = {fit_params[ii*3]} counts")
        print(f"Center{ii+1} = {fit_params[ii*3+1]} pixels")
        print(f"Full width half max{ii+1} = {fit_params[ii*3+2]} pixels")

    if plot:
        plt.plot_line_fit(intensity_l, gaussian_2_peak, fit_params)
    return fit_params

line(id_c[140])


[2.38338277e+04 3.45131494e+01 3.26420243e-01 8.38033093e+04
 4.88439189e+01 2.07448140e+00]
Amplitude1 = 23833.827722242197 counts
Center1 = 34.51314941405996 pixels
Full width half max1 = 0.3264202434169318 pixels
Amplitude2 = 83803.30928195185 counts
Center2 = 48.84391890276079 pixels
Full width half max2 = 2.0744814023009255 pixels


array([2.38338277e+04, 3.45131494e+01, 3.26420243e-01, 8.38033093e+04,
       4.88439189e+01, 2.07448140e+00])

In [ ]:
def find_peaks(intensity, cutoff):
    intensity_l = np.sum(intensity, axis=1)
    z_px = np.arange(len(intensity_l))
    above = np.where(intensity_l > cutoff)[0]
    print(above)
    print(np.diff(above))
    indices = np.where(np.diff(above) > 1)[0]
    print(indices)
    for index in indices:
        print(intensity_l[above[:index+1]])
        print(intensity_l[above[index+1:]])
find_peaks(id_c[140], 300)
    